### Importing Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates
import geopy
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

### Scraping the neighborhoods of delhi

In [2]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Delhi").text

In [3]:
soup = BeautifulSoup(data, 'lxml')

In [4]:
neighborhoodList = []

In [5]:
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [6]:
neighborhoodList.remove(neighborhoodList[0])

In [7]:
neighborhoodList

['Ashok Nagar (Delhi)',
 'Ashok Vihar',
 'Ashram Chowk',
 'Babarpur',
 'Badarpur, Delhi',
 'Bali Nagar',
 'Bawana',
 'Ber Sarai',
 'Bhajanpura',
 'Chanakyapuri',
 'Chandni Chowk',
 'Civil Lines, Delhi',
 'Connaught Place, New Delhi',
 'Dabri, New Delhi',
 'Dariba Kalan',
 'Daryaganj',
 'Dashrath Puri',
 'Dayanand Colony',
 'Defence Colony',
 'Delhi Cantonment',
 'Derawal Nagar',
 'Dhaula Kuan',
 'Dilshad Colony',
 'Dilshad Garden',
 'Dwarka, Delhi',
 'East Patel Nagar',
 'Gadaipur, Mehrauli, New Delhi',
 'Geetanjali Enclave',
 'Ghitorni',
 'Gole Market',
 'Golf Links, New Delhi',
 'Govindpuri',
 'Greater Kailash',
 'Green Park, Delhi',
 'Gulabi Bagh',
 'Gulmohar Park',
 'Hauz Khas',
 'Inder Puri',
 'Janakpuri',
 'Jangpura',
 'Jia Sarai',
 'Kabir Nagar, New Delhi',
 'Kailash Colony',
 'Kamla Nagar, New Delhi',
 'Kapasheda Border, Delhi',
 'Karol Bagh',
 'Keshav Puram',
 'Khaira, Delhi',
 'Khanpur, Delhi',
 'Khari Baoli',
 'Kingsway Camp',
 'Kirti Nagar',
 'Kotla Mubarakpur Complex',
 'K

### Creating a Dataframe

In [8]:
dl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

dl_df.head()

,Neighborhood
0,Ashok Nagar (Delhi)
1,Ashok Vihar
2,Ashram Chowk
3,Babarpur
4,"Badarpur, Delhi"


### Getting Geographical Coordinates

In [9]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    locator = Nominatim(user_agent="myGeocoder")
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g =locator.geocode('{}, Delhi, India'.format(neighborhood))
        if(g is None):
            g=geopy.geocoders.ArcGIS().geocode('{}, Delhi, India'.format(neighborhood))
        lat_lng_coords = [g.latitude,g.longitude]
    return lat_lng_coords

In [10]:
coords = [ get_latlng(neighborhood) for neighborhood in dl_df["Neighborhood"].tolist() ]

In [11]:
coords

[[28.6360207, 77.101822],
 [28.6994533, 77.1848256],
 [28.68313415, 77.19008284852976],
 [28.6874305, 77.2797551],
 [28.4931698, 77.3030239],
 [28.6541384, 77.128178],
 [28.79966, 77.0328847],
 [28.54843765, 77.18147936338744],
 [28.701012, 77.2609711],
 [28.5946775, 77.1885212],
 [28.6559834, 77.2321937],
 [28.6768508, 77.2250299],
 [28.6313827, 77.2197924],
 [28.6107697, 77.09110590789903],
 [28.6546024, 77.2333794],
 [28.64609, 77.243048],
 [28.605016, 77.0837828],
 [28.5621997, 77.2476131],
 [28.5712216, 77.23177637002767],
 [28.593833099999998, 77.13497891459929],
 [28.6997662, 77.19006005600144],
 [28.5918905, 77.1617034],
 [28.684700000000078, 77.32774000000006],
 [28.6758257, 77.3215163],
 [28.5771915, 77.0442935],
 [28.645649, 77.1725666],
 [28.4740326, 77.1730927],
 [28.5303586, 77.2020662],
 [28.493751, 77.1491866],
 [28.63371865, 77.20562662098214],
 [28.5959698, 77.2311628],
 [28.5351561, 77.2637936],
 [28.5591566, 77.2460179],
 [28.5555374, 77.2024966],
 [28.6696495, 77.1

In [12]:
dl_df['Latitude']=[x[0] for x in coords]
dl_df['Longitude']=[x[1] for x in coords]

In [13]:
dl_df

,Neighborhood,Latitude,Longitude
0,Ashok Nagar (Delhi),28.636021,77.101822
1,Ashok Vihar,28.699453,77.184826
2,Ashram Chowk,28.683134,77.190083
3,Babarpur,28.687431,77.279755
4,"Badarpur, Delhi",28.493170,77.303024
5,Bali Nagar,28.654138,77.128178
6,Bawana,28.799660,77.032885
7,Ber Sarai,28.548438,77.181479
8,Bhajanpura,28.701012,77.260971
9,Chanakyapuri,28.594677,77.188521


In [14]:
dl_df.to_csv('Delhi.csv',index=False)

### Creating a map of Delhi having with a neighbourhood marking on it

In [16]:
geolocation=Nominatim(user_agent='My geocoder')
location=geolocation.geocode('Delhi , India')
latitude=location.latitude
longitude=location.longitude
print(f'The geograpical coordinate of Kuala Lumpur, Malaysiae {latitude}, {longitude}.')

The geograpical coordinate of Kuala Lumpur, Malaysiae 28.6517178, 77.2219388.


In [18]:
# create map of Toronto using latitude and longitude values
map_dl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(dl_df['Latitude'], dl_df['Longitude'], dl_df['Neighborhood']):
    label = f'{neighborhood}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_dl)  
    
map_dl

In [19]:
map_dl.save('map_dl.html')

In [20]:
CLIENT_ID = '5VPISYFYORR5FMTRXK1B3T1QN5UXBOZ5SKAPVBN1KLFDMJXX' # your Foursquare ID
CLIENT_SECRET = '1IVUTKUMHAPJPFLAN1VZZR2NZRFYVFC5IXBTTFHYMEFR0UFK' # your Foursquare Secret
VERSION = '20200828' # Foursquare API version


In [21]:
radius = 2500
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(dl_df['Latitude'], dl_df['Longitude'], dl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [22]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(7439, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Ashok Nagar (Delhi),28.636021,77.101822,Zara | जा़रा,28.642468,77.106536,Clothing Store
1,Ashok Nagar (Delhi),28.636021,77.101822,PVR Cinemas,28.642439,77.106500,Multiplex
2,Ashok Nagar (Delhi),28.636021,77.101822,Pacific Mall,28.642334,77.106504,Shopping Mall
3,Ashok Nagar (Delhi),28.636021,77.101822,Manjeet Chicken,28.631452,77.097938,Fried Chicken Joint
4,Ashok Nagar (Delhi),28.636021,77.101822,Haldiram's,28.642459,77.106502,Indian Restaurant


### To see the number of venues near each neighborhood

In [23]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Ashok Nagar (Delhi),62,62,62,62,62,62
Ashok Vihar,39,39,39,39,39,39
Ashram Chowk,60,60,60,60,60,60
Babarpur,9,9,9,9,9,9
"Badarpur, Delhi",13,13,13,13,13,13
Bali Nagar,76,76,76,76,76,76
Bawana,1,1,1,1,1,1
Ber Sarai,93,93,93,93,93,93
Bhajanpura,2,2,2,2,2,2


### To print no. of unique venue categories 

In [26]:
length=len(venues_df['VenueCategory'].unique())
print(f'There are {length} uniques categories')

There are 213 uniques categories


### One hot encoding

In [27]:
dl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dl_onehot.columns[-1]] + list(dl_onehot.columns[:-1])
dl_onehot = dl_onehot[fixed_columns]

print(dl_onehot.shape)
dl_onehot.head()

(7439, 214)


,Neighborhoods,ATM,Accessories Store,Airport,Airport Food Court,Airport Service,Airport Terminal,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Bridal Shop,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Candy Store,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Luggage Store,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Neighborhood,Nightclub,Nightlife Spot,North Indian Restaurant,Northeast Indian Restaurant,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Punjabi Restaurant,Racetrack,Resort,Restaurant,River,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Udupi Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Whisky Bar,Women's Store,Yoga Studio,Zoo
0,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [31]:
dl_grouped = dl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(dl_grouped.shape)
dl_grouped

(138, 214)


,Neighborhoods,ATM,Accessories Store,Airport,Airport Food Court,Airport Service,Airport Terminal,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Bridal Shop,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Candy Store,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Luggage Store,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Neighborhood,Nightclub,Nightlife Spot,North Indian Restaurant,Northeast Indian Restaurant,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Punjabi Restaurant,Racetrack,Resort,Restaurant,River,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Udupi Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Whisky Bar,Women's Store,Yoga Studio,Zoo
0,Ashok Nagar (Delhi),0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.016129,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.048387,0.000000,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.080645,0.000000,0.000000,0.016129,0.032258,0.000000,0.064516,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.016129,0.000000,0.016129,0.000000,0.048387,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.129032,0.000000,0.00,0.000000,0.000000,0.016129,0.000000,0.000000,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016129,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.032258,0.000000,0.129032,0.016129,0.000000,0.00,0.016129,0.000000,0.000000,0.000000,0.000000,0.00,0

In [32]:
len(dl_grouped[dl_grouped["Shopping Mall"] > 0])

46

#### Creating a dataframe for shopping mall only

In [33]:
dl_mall = dl_grouped[["Neighborhoods","Shopping Mall"]]

In [34]:
dl_mall.head()

,Neighborhoods,Shopping Mall
0,Ashok Nagar (Delhi),0.032258
1,Ashok Vihar,0.000000
2,Ashram Chowk,0.000000
3,Babarpur,0.000000
4,"Badarpur, Delhi",0.000000


## Clustering

In [35]:
kclusters = 3

dl_clustering = dl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dl_clustering)

In [39]:
dl_new = dl_mall.copy()

dl_new['Cluster']=kmeans.labels_

dl_new

,Neighborhoods,Shopping Mall,Cluster
0,Ashok Nagar (Delhi),0.032258,1
1,Ashok Vihar,0.000000,0
2,Ashram Chowk,0.000000,0
3,Babarpur,0.000000,0
4,"Badarpur, Delhi",0.000000,0
5,Bali Nagar,0.052632,1
6,Bawana,0.000000,0
7,Ber Sarai,0.010753,0
8,Bhajanpura,0.000000,0
9,Chanakyapuri,0.012346,0


In [44]:
dl_new = dl_new.join(dl_df.set_index("Neighborhood"), on="Neighborhoods")

print(dl_new.shape)
dl_new.head()

(138, 5)


,Neighborhoods,Shopping Mall,Cluster,Latitude,Longitude
0,Ashok Nagar (Delhi),0.032258,1,28.636021,77.101822
1,Ashok Vihar,0.000000,0,28.699453,77.184826
2,Ashram Chowk,0.000000,0,28.683134,77.190083
3,Babarpur,0.000000,0,28.687431,77.279755
4,"Badarpur, Delhi",0.000000,0,28.493170,77.303024


In [46]:
dl_new.sort_values(["Cluster"], inplace=True)
dl_new

,Neighborhoods,Shopping Mall,Cluster,Latitude,Longitude
68,Mehrauli,0.000000,0,28.521826,77.178323
87,Nizamuddin West,0.000000,0,28.588365,77.244955
86,Nizamuddin East,0.000000,0,28.589667,77.250385
85,Nigambodh Ghat,0.000000,0,28.664710,77.236330
84,New Moti Bagh,0.000000,0,28.667619,77.184227
83,New Friends Colony,0.000000,0,28.567101,77.269764
82,New Delhi,0.000000,0,28.613895,77.209006
81,"Netaji Nagar, Delhi",0.010309,0,28.575582,77.186163
80,Naveen Shahdara,0.000000,0,28.677339,77.286016
79,Narela,0.000000,0,28.842610,77.091835


In [50]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dl_new['Latitude'], dl_new['Longitude'], dl_new['Neighborhoods'], dl_new['Cluster']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [51]:
dl_new[dl_new['Cluster']==0]

,Neighborhoods,Shopping Mall,Cluster,Latitude,Longitude
68,Mehrauli,0.000000,0,28.521826,77.178323
87,Nizamuddin West,0.000000,0,28.588365,77.244955
86,Nizamuddin East,0.000000,0,28.589667,77.250385
85,Nigambodh Ghat,0.000000,0,28.664710,77.236330
84,New Moti Bagh,0.000000,0,28.667619,77.184227
83,New Friends Colony,0.000000,0,28.567101,77.269764
82,New Delhi,0.000000,0,28.613895,77.209006
81,"Netaji Nagar, Delhi",0.010309,0,28.575582,77.186163
80,Naveen Shahdara,0.000000,0,28.677339,77.286016
79,Narela,0.000000,0,28.842610,77.091835


In [52]:
dl_new[dl_new['Cluster']==1]

,Neighborhoods,Shopping Mall,Cluster,Latitude,Longitude
109,Saket (Delhi),0.040000,1,28.524411,77.213725
110,Saket District Centre,0.040000,1,28.528101,77.219035
128,Tilak Nagar (Delhi),0.037037,1,28.639650,77.094039
120,Shankar Vihar,0.040000,1,28.557439,77.139665
126,South Patel Nagar,0.025000,1,28.644474,77.168466
124,Shivaji Place,0.044776,1,28.651657,77.121703
117,Shahpur Jat,0.020000,1,28.548330,77.214104
131,Vasant Kunj,0.050847,1,28.529249,77.154134
5,Bali Nagar,0.052632,1,28.654138,77.128178
116,Shahdara district,0.040816,1,28.653900,77.296410


In [54]:
dl_new[dl_new['Cluster']==2]

,Neighborhoods,Shopping Mall,Cluster,Latitude,Longitude
55,Laxmi Nagar (Delhi),0.076923,2,28.630553,77.277575
104,"Rohini, Delhi",0.096774,2,28.716209,77.117074
19,Delhi Cantonment,0.066667,2,28.593833,77.134979
95,Patel Nagar,0.062500,2,28.659809,77.156957
46,Keshav Puram,0.075000,2,28.688926,77.161683
119,Shalimar Bagh (Delhi Assembly constituency),0.111111,2,28.714230,77.157440
96,Pitam Pura,0.090909,2,28.703268,77.132250


So as we can see that the cluster of neighborhoods which is labeled as 2 has the highest numbers of shopping malls around the area of radius 2.5 km while the while neighborhoods labeled as 1 has moderate numbers of shooping malls and neighborhoods labeled as 1 has very lesser numbers of malls or has no shopping mall aroung 2.5km radius. As we know cluster 2 neighborhoods has higher number of shopping malls hence they are suffering from intense competition amongst each other while there is comparatively less competiton in cluster 1 neighborhoods and cluster 0 neighborhoods has almost no competition .Hence it is recommended by this project that propety developer should open a shopping mall in the neighborhood which is labeled as 0 as there is less number of other shopping mall due to which there is less competition and also the people living in nearby area will have to travel less for the shopping mall.